In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from transformers import create_optimizer
from transformers import TFDebertaForSequenceClassification,DebertaTokenizerFast
import tensorflow as tf

/Users/suyash/Desktop/Intent-classification-/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


RuntimeError: Failed to import transformers.optimization_tf because of the following error (look up to see its traceback):
Your currently installed version of Keras is Keras 3, but this is not yet supported in Transformers. Please install the backwards-compatible tf-keras package with `pip install tf-keras`.

In [ ]:
from datasets import load_dataset

In [ ]:
datasets = load_dataset(
    "csv",
    data_files="/content/dataset/20000-Utterances-Training-dataset-for-chatbots-virtual-assistant-Bitext-sample/20000-Utterances-Training-dataset-for-chatbots-virtual-assistant-Bitext-sample/20000-Utterances-Training-dataset-for-chatbots-virtual-assistant-Bitext-sample.csv"
)

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
datasets

DatasetDict({
    train: Dataset({
        features: ['flags', 'utterance', 'category', 'intent'],
        num_rows: 21534
    })
})

In [ ]:
datasets['train'][0]

{'flags': 'BILC',
 'utterance': "I don't have an online account, what do I have to do to register?",
 'category': 'ACCOUNT',
 'intent': 'create_account'}

In [ ]:
intents=list(set(datasets['train']['intent']))
dict_intents={intents[i]: i for i in range(len(intents))}
print(dict_intents)
print(len(intents))

{'contact_customer_service': 0, 'review': 1, 'check_invoices': 2, 'recover_password': 3, 'switch_account': 4, 'get_invoice': 5, 'complaint': 6, 'payment_issue': 7, 'check_payment_methods': 8, 'delivery_period': 9, 'change_order': 10, 'set_up_shipping_address': 11, 'place_order': 12, 'cancel_order': 13, 'track_order': 14, 'delivery_options': 15, 'check_cancellation_fee': 16, 'change_shipping_address': 17, 'contact_human_agent': 18, 'track_refund': 19, 'create_account': 20, 'registration_problems': 21, 'delete_account': 22, 'get_refund': 23, 'newsletter_subscription': 24, 'check_refund_policy': 25, 'edit_account': 26}
27


In [ ]:
len(set(datasets['train']['intent']))

27

In [ ]:
def preprocess(dataset):
  return {'utterance':dataset['utterance'],
          'intent':dict_intents[dataset['intent']]}

In [ ]:
prep_dataset=datasets.map(preprocess)

Map:   0%|          | 0/21534 [00:00<?, ? examples/s]

In [ ]:
prep_dataset['train'][0]

{'flags': 'BILC',
 'utterance': "I don't have an online account, what do I have to do to register?",
 'category': 'ACCOUNT',
 'intent': 20}

In [ ]:
model_id="microsoft/deberta-base"
tokenizer = DebertaTokenizerFast.from_pretrained(model_id)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/474 [00:00<?, ?B/s]

In [ ]:
def tokenizer_function(dataset):
  return tokenizer(dataset["utterance"],)

In [ ]:
tokenized_dataset=prep_dataset.map(tokenizer_function)

Map:   0%|          | 0/21534 [00:00<?, ? examples/s]

In [ ]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['flags', 'utterance', 'category', 'intent', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 21534
    })
})

In [ ]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

In [ ]:
tf_dataset = tokenized_dataset["train"].to_tf_dataset(
    columns=['input_ids','attention_mask', 'intent'],
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator
)

In [ ]:
def swap_positions(dataset):
  return {'input_ids':dataset['input_ids'],
          'attention_mask':dataset['attention_mask'],},dataset['intent']

In [ ]:
tf_dataset=tf_dataset.map(swap_positions)

In [ ]:
train_dataset=tf_dataset.take(int(0.9*len(tf_dataset)))
val_dataset=tf_dataset.skip(int(0.9*len(tf_dataset)))

In [ ]:
model=TFDebertaForSequenceClassification.from_pretrained(model_id,num_labels=len(intents))
model.summary()

tf_model.h5:   0%|          | 0.00/555M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFDebertaForSequenceClassification.

Some layers of TFDebertaForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-base and are newly initialized: ['pooler', 'classifier', 'cls_dropout']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model: "tf_deberta_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 deberta (TFDebertaMainLaye  multiple                  138601728 
 r)                                                              
                                                                 
 pooler (TFDebertaContextPo  multiple                  590592    
 oler)                                                           
                                                                 
 cls_dropout (TFDebertaStab  multiple                  0         
 leDropout)                                                      
                                                                 
 classifier (Dense)          multiple                  20763     
                                                                 
Total params: 139213083 (531.06 MB)
Trainable params: 139213083 (531.06 MB)
Non-trainable para

In [ ]:
num_epochs = 2
batches_per_epoch = len(tokenized_dataset["train"]) // 16
total_train_steps = int(batches_per_epoch * num_epochs)

In [ ]:
optimizer, schedule = create_optimizer(init_lr=2e-5,num_warmup_steps=0, num_train_steps=total_train_steps)

In [ ]:
model.compile(optimizer=optimizer,
    metrics=["accuracy"])

In [ ]:
history=model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=2,)

Epoch 1/2


Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.


1211/1211 [==============================] - 292s 184ms/step - loss: 0.2553 - accuracy: 0.9445 - val_loss: 0.0048 - val_accuracy: 0.9995
Epoch 2/2
1211/1211 [==============================] - 213s 176ms/step - loss: 0.0107 - accuracy: 0.9983 - val_loss: 0.0014 - val_accuracy: 1.0000


In [ ]:
model.save('intent', save_format='tf')

In [ ]:
inputs = tokenizer(["Please how do i go about the account creation? ",
                    "After setting up my account, i feel like i need to change it. How do i go about that?",
                    "how do i know how much i need to pay?",
                    "purchased a product, which i now want to change"
                    ], padding=True,return_tensors="tf")

logits = model(**inputs).logits
outputs=tf.argmax(logits,axis=-1).numpy()

In [ ]:
print(outputs)

[20  4  8 10]


In [ ]:
reverse_dict_intents={i:intents[i] for i in range(len(intents))}
print(reverse_dict_intents)

{0: 'contact_customer_service', 1: 'review', 2: 'check_invoices', 3: 'recover_password', 4: 'switch_account', 5: 'get_invoice', 6: 'complaint', 7: 'payment_issue', 8: 'check_payment_methods', 9: 'delivery_period', 10: 'change_order', 11: 'set_up_shipping_address', 12: 'place_order', 13: 'cancel_order', 14: 'track_order', 15: 'delivery_options', 16: 'check_cancellation_fee', 17: 'change_shipping_address', 18: 'contact_human_agent', 19: 'track_refund', 20: 'create_account', 21: 'registration_problems', 22: 'delete_account', 23: 'get_refund', 24: 'newsletter_subscription', 25: 'check_refund_policy', 26: 'edit_account'}


In [ ]:
for i in outputs:
  print(reverse_dict_intents[i])

create_account
switch_account
check_payment_methods
change_order


In [ ]:
tokenizer.save_pretrained("./saved_model2")

('./saved_model2/tokenizer_config.json',
 './saved_model2/special_tokens_map.json',
 './saved_model2/vocab.json',
 './saved_model2/merges.txt',
 './saved_model2/added_tokens.json',
 './saved_model2/tokenizer.json')

In [ ]:
# prompt: save the model

model.save_pretrained('./saved_model')
tokenizer.save_pretrained("./saved_model")

('./saved_model/tokenizer_config.json',
 './saved_model/special_tokens_map.json',
 './saved_model/vocab.json',
 './saved_model/merges.txt',
 './saved_model/added_tokens.json',
 './saved_model/tokenizer.json')

In [ ]:
# prompt: make a interfaces with the gradio

import gradio as gr
import tensorflow as tf
from transformers import TFDebertaForSequenceClassification, DebertaTokenizerFast

# Load the saved model and tokenizer using from_pretrained
model = TFDebertaForSequenceClassification.from_pretrained('/content/saved_model')
tokenizer = DebertaTokenizerFast.from_pretrained("/content/saved_model")

# Reverse dictionary to map output to intents
reverse_dict_intents = {0: 'contact_customer_service', 1: 'review', 2: 'check_invoices', 3: 'recover_password', 4: 'switch_account', 5: 'get_invoice', 6: 'complaint', 7: 'payment_issue', 8: 'check_payment_methods', 9: 'delivery_period', 10: 'change_order', 11: 'set_up_shipping_address', 12: 'place_order', 13: 'cancel_order', 14: 'track_order', 15: 'delivery_options', 16: 'check_cancellation_fee', 17: 'change_shipping_address', 18: 'contact_human_agent', 19: 'track_refund', 20: 'create_account', 21: 'registration_problems', 22: 'delete_account', 23: 'get_refund', 24: 'newsletter_subscription', 25: 'check_refund_policy', 26: 'edit_account'}

def predict_intent(text):
    inputs = tokenizer([text], padding=True, return_tensors="tf")
    logits = model(**inputs).logits
    output = tf.argmax(logits, axis=-1).numpy()[0]
    return reverse_dict_intents[output]

# Use gr.Textbox directly instead of gr.inputs.Textbox
iface = gr.Interface(
    fn=predict_intent,
    inputs=gr.Textbox(lines=2, placeholder="Enter your text here..."), # Changed this line
    outputs="text",
    title="Intent Classification",
    description="Enter a text query and get its predicted intent.",
)

iface.launch()

All model checkpoint layers were used when initializing TFDebertaForSequenceClassification.

All the layers of TFDebertaForSequenceClassification were initialized from the model checkpoint at /content/saved_model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDebertaForSequenceClassification for predictions without further training.


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://e3969f9fca293d11d2.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
import gradio as gr
import tensorflow as tf
from transformers import TFDebertaForSequenceClassification, DebertaTokenizerFast

# Load the saved model and tokenizer using from_pretrained
model = TFDebertaForSequenceClassification.from_pretrained('/content/saved_model')
tokenizer = DebertaTokenizerFast.from_pretrained("/content/saved_model")

# Reverse dictionary to map output to intents
reverse_dict_intents = {0: 'contact_customer_service', 1: 'review', 2: 'check_invoices', 3: 'recover_password', 4: 'switch_account', 5: 'get_invoice', 6: 'complaint', 7: 'payment_issue', 8: 'check_payment_methods', 9: 'delivery_period', 10: 'change_order', 11: 'set_up_shipping_address', 12: 'place_order', 13: 'cancel_order', 14: 'track_order', 15: 'delivery_options', 16: 'check_cancellation_fee', 17: 'change_shipping_address', 18: 'contact_human_agent', 19: 'track_refund', 20: 'create_account', 21: 'registration_problems', 22: 'delete_account', 23: 'get_refund', 24: 'newsletter_subscription', 25: 'check_refund_policy', 26: 'edit_account'}

def predict_intent(text):
    inputs = tokenizer([text], padding=True, return_tensors="tf")
    logits = model(**inputs).logits
    output = tf.argmax(logits, axis=-1).numpy()[0]
    return reverse_dict_intents[output]

# Create Gradio Interface
iface = gr.Interface(
    fn=predict_intent,
    inputs=gr.Textbox(lines=2, placeholder="Enter your text here..."),
    outputs="text",
    title="Intent Classification",
    description="Enter a text query and get its predicted intent.",
)

# Launch interface with flag='never' to avoid external sharing
iface.launch(share=False)


All model checkpoint layers were used when initializing TFDebertaForSequenceClassification.

All the layers of TFDebertaForSequenceClassification were initialized from the model checkpoint at /content/saved_model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDebertaForSequenceClassification for predictions without further training.


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>